List the stations in batches to download the data

In [4]:
import pandas as pd

# Load the file
df = pd.read_csv("AB_station_list.csv", header=None)

# Convert to list of station IDs (strings)
stations = df[0].astype(str).tolist()

# Group into chunks of 50 IDs
chunks = [stations[i:i+100] for i in range(0, len(stations), 100)]

# Convert each chunk back into a comma-separated string
chunk_strings = [",".join(chunk) for chunk in chunks]

for line in chunk_strings:
    print(line)

05AA001,05AA002,05AA003,05AA004,05AA005,05AA006,05AA007,05AA008,05AA009,05AA010,05AA011,05AA013,05AA014,05AA015,05AA016,05AA021,05AA022,05AA023,05AA024,05AA025,05AA026,05AA027,05AA028,05AA029,05AA030,05AA032,05AA033,05AA034,05AA035,05AA909,05AB002,05AB005,05AB006,05AB007,05AB011,05AB013,05AB014,05AB015,05AB016,05AB017,05AB018,05AB019,05AB020,05AB021,05AB022,05AB023,05AB024,05AB025,05AB026,05AB027,05AB028,05AB029,05AB030,05AB031,05AB032,05AB035,05AB036,05AB037,05AB038,05AB039,05AB040,05AB041,05AB042,05AB044,05AB045,05AB046,05AC001,05AC002,05AC003,05AC004,05AC005,05AC006,05AC009,05AC010,05AC011,05AC012,05AC013,05AC014,05AC015,05AC016,05AC017,05AC022,05AC023,05AC024,05AC025,05AC026,05AC027,05AC028,05AC029,05AC030,05AC031,05AC032,05AC033,05AC034,05AC901,05AC902,05AC909,05AC911,05AC917,05AC918
05AC919,05AC921,05AC922,05AC924,05AC926,05AC927,05AC928,05AC929,05AC930,05AC937,05AC938,05AC939,05AC940,05AC941,05AD001,05AD002,05AD003,05AD004,05AD005,05AD006,05AD007,05AD008,05AD010,05AD012,05AD013,

Combine the raw data files into a usable format

In [17]:
import pandas as pd
import glob
import os

folder = "raw_streamflow_batch_data"

# Load all daily_data (#).csv files
files = sorted(glob.glob(os.path.join(folder, "daily_data (*.csv")))

all_data = []

for f in files:
    df = pd.read_csv(f)

    # Keep only needed columns
    df = df[[" ID", "Date", "Value/Valeur"]]

    all_data.append(df)

# Combine all files into a single long-format DataFrame
df_long = pd.concat(all_data, ignore_index=True)

# Convert Date to datetime to ensure proper sorting
df_long["Date"] = pd.to_datetime(df_long["Date"])

# Pivot into wide format: rows = dates, columns = station IDs
df_wide = df_long.pivot(index="Date", columns=" ID", values="Value/Valeur")

# Sort rows (by Date) and columns (station IDs)
df_wide = df_wide.sort_index().sort_index(axis=1)

df_wide.to_csv("combined_streamflow.csv")


Trying to automate this a bit better

In [20]:
import urllib.parse

def build_wateroffice_url(stations, start_date, end_date, parameter="flow"):
    """
    Build a Wateroffice batch-download URL for daily data.
    
    Parameters
    ----------
    stations : list of str
        List of hydrometric station numbers (e.g. ["11AB104", "11AB105"]).
    start_date : str
        Start date in YYYY-MM-DD.
    end_date : str
        End date in YYYY-MM-DD.
    parameter : str
        Usually "flow" for discharge.
    
    Returns
    -------
    str
        A full URL that can be used to download a combined CSV of all stations.
    """

    base = "https://wateroffice.ec.gc.ca/services/daily_data/csv/inline?"

    # Encode station parameters properly
    # stations[]=11AB104&stations[]=11AB105&...
    station_params = "&".join([f"stations[]={urllib.parse.quote(s)}" for s in stations])

    # Encode parameter (e.g. flow)
    param_part = f"parameters[]={urllib.parse.quote(parameter)}"

    # Add date range
    date_part = f"start_date={start_date}&end_date={end_date}"

    url = base + station_params + "&" + param_part + "&" + date_part
    return url


In [21]:
stations = [
    "11AB104", "11AB105", "11AB107",
    "11AB111", "11AB117", "11AB902"
]

url = build_wateroffice_url(
    stations,
    start_date="1980-01-01",
    end_date="2022-12-31"
)

print(url)

https://wateroffice.ec.gc.ca/services/daily_data/csv/inline?stations[]=11AB104&stations[]=11AB105&stations[]=11AB107&stations[]=11AB111&stations[]=11AB117&stations[]=11AB902&parameters[]=flow&start_date=1980-01-01&end_date=2022-12-31


In [22]:
def download_wateroffice_data(stations, start_date, end_date, parameter="flow"):
    url = build_wateroffice_url(stations, start_date, end_date, parameter)
    return pd.read_csv(url)

stations = [
    "11AB104", "11AB105", "11AB107",
    "11AB111", "11AB117", "11AB902"
]

df = download_wateroffice_data(
    stations,
    start_date="1980-01-01",
    end_date="2022-12-31"
)

df.head()

,ID,Date,Parameter/Paramètre,Value/Valeur,Symbol/Symbole
0,11AB117,1980-02-25,discharge/débit,0.114,B
1,11AB117,1980-02-26,discharge/débit,0.119,B
2,11AB117,1980-02-27,discharge/débit,0.300,B
3,11AB117,1980-02-28,discharge/débit,0.250,B
4,11AB117,1980-02-29,discharge/débit,0.220,B


Full pipeline using the metadata

In [ ]:
import urllib.parse
import pandas as pd

batch_size = 50

# define start and end years
start_year = 1980
end_year = 2022

metadata = pd.read_csv("station_metadata.csv")

def build_wateroffice_url(stations, start_date, end_date, parameter="flow"):
    """
    Build a Wateroffice batch-download URL for daily data.
    
    Parameters
    ----------
    stations : list of str
        List of hydrometric station numbers (e.g. ["11AB104", "11AB105"]).
    start_date : str
        Start date in YYYY-MM-DD.
    end_date : str
        End date in YYYY-MM-DD.
    parameter : str
        Usually "flow" for discharge.
    
    Returns
    -------
    str
        A full URL that can be used to download a combined CSV of all stations.
    """

    base = "https://wateroffice.ec.gc.ca/services/daily_data/csv/inline?"

    # Encode station parameters properly
    # stations[]=11AB104&stations[]=11AB105&...
    station_params = "&".join([f"stations[]={urllib.parse.quote(s)}" for s in stations])

    # Encode parameter (e.g. flow)
    param_part = f"parameters[]={urllib.parse.quote(parameter)}"

    # Add date range
    date_part = f"start_date={start_date}&end_date={end_date}"

    url = base + station_params + "&" + param_part + "&" + date_part
    return url

def download_wateroffice_data(stations, start_date, end_date, parameter="flow"):
    url = build_wateroffice_url(stations, start_date, end_date, parameter)
    return pd.read_csv(url)

# filter the stations based on given year range
filtered_metadata = metadata[(metadata['Year From'] <= start_year) & (metadata['Year To'] >= end_year)]
study_stations = filtered_metadata["Station Number"].tolist()

all_data = []

for i in range(0, len(study_stations), batch_size):
    batch_stations = study_stations[i:i + batch_size]
    
    df_batch = download_wateroffice_data(
        batch_stations,
        start_date=f"{start_year}-01-01",
        end_date=f"{end_year}-12-31"
    )

    all_data.append(df_batch[[" ID", "Date", "Value/Valeur"]])
    
    # # Save each batch to a separate CSV file
    # batch_number = i // batch_size + 1
    # df_batch.to_csv(f"raw_streamflow_batch_data/daily_data ({batch_number}).csv", index=False)

# Combine all files into a single long-format DataFrame
df_long = pd.concat(all_data, ignore_index=True)

# Convert Date to datetime to ensure proper sorting
df_long["Date"] = pd.to_datetime(df_long["Date"])

# Pivot into wide format: rows = dates, columns = station IDs
df_wide = df_long.pivot(index="Date", columns=" ID", values="Value/Valeur")

# Sort rows (by Date) and columns (station IDs)
df_wide = df_wide.sort_index().sort_index(axis=1)

df_wide.to_csv("combined_streamflow.csv")

In [36]:
df_wide.shape

(15706, 180)